# Creating L3 VPN Network connecting FABRIC with Cloud

## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import json

try:
    fablib = fablib_manager()
                     
    fablib.show_config();
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
from plugins import Plugins
import traceback
try:
    Plugins.load()
except Exception as e:
    traceback.print_exc()

## Create the Experiment Slice

The following creates 
- A node on a site with Basic NIC and connects it to a FABRIC L3VPN Service
- Cloud Facility port and connects it to a Cloud L3VPN Service
- Peers Cloud and FABRIC L3VPN Services

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

slice_name = 'MySlice-al2s-3'
site1='CERN'

node1_name = 'Node1'

network1_name='net1'

node1_nic_name = 'nic1'

al2s_network_name = 'al2s'

model = 'NIC_Basic'
#model = 'NIC_ConnectX_5'


In [ ]:
from fim.slivers.capacities_labels import Labels, Capacities
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Create a VM on a site
    node1 = slice.add_node(name=node1_name, site=site1)
    iface1 = node1.add_component(model=model, name=node1_nic_name).get_interfaces()[0]
    fim_iface1 = iface1.get_fim_interface()
    fim_iface1.labels = Labels.update(fim_iface1.labels, ipv4_subnet=f'192.168.10.1/24')
    
    # Create FABRIC L3VPN and connect to the VM
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='L3VPN')

    # Create a Cloud Facility Port
    fabric_facility_port = slice.add_facility_port(name='Cloud-Facility-AWS', site='AWS',
                                                   # Facility Port can be chosen either by passing region, local_name or device_name, if neither is passed a random Port is returned
                                                   # VLAN can be user specified or can be assigned by ControlFramework if not specified
                                                   
                                                   # Only Specify Port Name
                                                   #labels=Labels(vlan='2', ipv4_subnet='192.168.30.1/24', local_name='HundredGigE0/0/0/7'),
                                                   
                                                   # Only specify region
                                                   #labels=Labels(ipv4_subnet='192.168.30.1/24', region='us-east-1'),
                                                   
                                                   # Only specify device name
                                                   labels=Labels(ipv4_subnet='192.168.30.1/24', device_name='agg3.dall3.net.internet2.edu'),
                                                   
                                                   # Device and Region Name
                                                   # labels=Labels(ipv4_subnet='192.168.30.1/24', region='us-east-1', device_name='agg4.ashb.net.internet2.edu'),
                                                   
                                                   # Device and Port Name
                                                   # labels=Labels(ipv4_subnet='192.168.30.1/24', device_name='agg4.ashb.net.internet2.edu', local_name='HundredGigE0/0/0/7'),
                                                   
                                                   # Region, Device and Port Name
                                                   #labels=Labels(ipv4_subnet='192.168.30.1/24', region='us-east-1', device_name='agg4.ashb.net.internet2.edu', local_name='HundredGigE0/0/0/7'),
                                                   
                                                   # Peer Labels
                                                   peer_labels=Labels(ipv4_subnet='192.168.30.2/24',asn='64512', 
                                                                      bgp_key='0xzsEwC7xk6c1fK_h.xHyAdx', 
                                                                      account_id='296256999979'),
                                                   # MTU must be set to 9001 for Cloud Facility Ports
                                                   capacities=Capacities(bw=1, mtu=9001))
    
    fabric_facility_port_iface = fabric_facility_port.get_interfaces()[0]
    
    # Create Cloud L3VPN and connect to Cloud Facility Port
    al2s = slice.add_l3network(name=al2s_network_name, interfaces=[fabric_facility_port_iface], type='L3VPN', technology='AL2S')

    # Peer Cloud L3VPN with FABRIC L3VPN
    al2s.fim_network_service.peer(net1.fim_network_service, 
                                  labels=Labels(bgp_key='secret', ipv4_subnet='192.168.50.1/24'),
                                  # bw must be set to 0 and mtu 9001 for Peered Interfaces
                                  capacities=Capacities(mtu=9001))
    
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")
    import traceback
    traceback.print_exc()

## Observe the Slice's Attributes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

## Interfaces for FABRIC L3VPN

In [ ]:
fabric_l3_vpn = slice.get_network(name=network1_name)

for x in fabric_l3_vpn.get_fim_network_service().interface_list:
    print("-----------------------")
    print(f"Interface Name: {x.name}")
    print(f"Capacities: {x.capacities}")
    print(f"Labels: {x.labels}")
    print(f"LabelAllocations: {x.get_property(pname='label_allocations')}")
    print(f"PeerLabels: {x.peer_labels}")
    print("-----------------------")

## Interfaces for AL2S L3VPN

In [ ]:
al2s_l3_vpn = slice.get_network(name=al2s_network_name)

for x in al2s_l3_vpn.get_fim_network_service().interface_list:
    print("-----------------------")
    print(f"Interface Name: {x.name}")
    print(f"Capacities: {x.capacities}")
    print(f"Labels: {x.labels}")
    print(f"LabelAllocations: {x.get_property(pname='label_allocations')}")
    print(f"PeerLabels: {x.peer_labels}")
    print("-----------------------")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")